In [1]:
import numpy as np

# Robbie el Robot Recogebasura

Este ejemplo proviene del libro {cite}`Mitchell2009` de [Melanie Mitchell](https://melaniemitchell.me/).

El capítulo 9 [aquí](http://modelai.gettysburg.edu/2016/robby/Chapter9_Complexity.pdf).

## Presentación

* _Mundo_ bidimensional: mallado de $10\times10$ celdas
* Robot Robbie
* Celdas con/sin lata/basura (con probabilidad)
* Recinto cerrado con __pared__

```{figure} ./images/ga_robbie_1.png
```

* Robbie es un robot que puede
   * ver lo que sucede en las celdas inmediatamente adyacentes (N,S,E,O)
   * ver lo que sucede en su celda
   * recoger lo que hay en su celda
   * moverse (N,S,E,O)
  
### Sesión de limpieza

* Cada sesión lleva 200 acciones de Robbie (recoger, moverse)
* Cada acción tiene un premio o castigo
   * Si Robbie está en un sitio con basura y la recoge: premio = 10 puntos
   * Si Robbie se agacha a recoger basura y no hay: castigo = 1 punto
   * Si Robbie se choca con una pared: castigo = 5 puntos (y rebota al sitio donde estaba)

__Objetivo:__ maximizar el premio de Robbie $\sim$ limpiar lo más posible el recinto sin chocarse ni agacharse a recoger cuando no hay basura.

> ¿Podemos pensar en una _estrategia_ para limpiar el recinto de la manera más eficiente posible?

## Estrategias

* Vamos a dejar que el ordenador evolucione una _estrategia_ para Robbie
* ¿_Estrategia_?
   * Conjunto de __normas__: para cada posible situación, la acción a tomar
   * _¿Situación?_: lo que Robbie puede ver (su posición y sus vecinos)
   * _¿Acción a tomar?_: elegir entre las 7 posibles acciones
        * Moverse (N,S,E,O, random)
        * Quedarse quieto
        * Recoger lata

> Ejemplo de una situación y una acción

| Situation                            || Action    |
|-------|-------|------|------|---------|-----------|
| North | South | East | West | Current |           |
|-------|-------|------|------|---------|-----------|
| Wall  | Empty | Can  | Wall | Empty   | Move West |

* Robbie tiene una _tabla_ con todas las posibles situaciones y las acciones a tomar en cada situación.
* La estrategia anterior __no__ es una buena estrategia, pero es __una__ estrategia

> Posibles situaciones: hay 243 situaciones posibles (como máximo). La lista debe contener al menos ese número de _entradas_.


## Algoritmo Genético

* Población: cada individuo es una _estrategia_: un listado de las acciones a seguir por Robbie en cada una de las 243 posibles situaciones.

Por ejemplo, un individuo/estrategia es:

> MoveNorth MoveEast MoveRandom PickUpCan ... MoveWest ... StayPut

* Las acciones están listadas siempre en el mismo orden, por lo tanto Robbie es capaz de reconocer la acción a tomar por su posición en la lista (situación).
   * No es necesario poner la lista de todas las situaciones una y otra vez: con las acciones basta.

### Fases

1. Población inicial: 200 estrategias/individuos aleatorias

```{figure} ./images/ga_robbie_2.png
```

* Cada estrategia individual es una lista de 243 _genes_
* Cada _gen_ es un número entre 0 y 6.
   * 0: MoveNorth
   * 1: MoveSouth
   * ...
   * 6: RandomMove

__Repetir para $N=1000$ generaciones__

2. Calcular el _fitness_ de cada individuo en la población

* Realizar 100 sesiones de limpieza con Robbie para cada estrategia
* Robbie empieza en la posición (0,0)
* Se colocan latas aleatoriamente en el mallado (cada sitio puede tener a lo sumo una lata)
   * La probabilidad de contener una lata es del 50%
* Cada sesión de limpieza son 200 acciones de Robbie
* Al final de la sesión se anota el puntaje de Robbie
* El _fitness_ de la estrategia es la media de los puntajes para las 100 sesiones

3. __Evolución__

* Crear una nueva población a partir de la ya existente

Repetir hasta obtener una población con $200$ individuos:

* (a) Elegir dos individuos (padres) de la población actual probabilísticamente basándose en el _fitness_
   * a mayor _fitness_, mayor probabilidad de ser elegidos
* (b) _Crear_ un _hijo_ a partir de los dos _padres_
   * elegir una posición aleatoria, $p$, para _partir_ las dos cadenas de genes
   * formar un _hijo_ tomando la cadena del padre $A$ hasta $p$ y la cadena del padre $B$ desde $p$
   * _viceversa_ para formar un segundo hijo
* (c) _Mutar_ números de los hijos formados: con una probabilidad dada (pequeña), elegir uno o más números de la cadena de los hijos y reemplazarlos por un número aleatorio entre el 0 y el 6
* (d) Incluír los hijos en la población

4. Repetir desde el paso 2.

## _Fitness score_

Si en cada sesión de limpieza (simulación) cada espacio tiene un $50\%$ de probabilidad de contener una lata, por tanto, aproximadamente el $50\%$ de los sitios tendrán una lata, es decir, habrá de media 50 latas en el recinto.

El máximo _score_ posible es _aproximadamente_ 500.


## Estrategia _propia_

Compararemos el resultado del GA con una estrategia _inteligente_:

* Si hay una lata en el sitio actual: recógela
* Si hay una lata en un sitio adyacente, muévete al sitio adyacente
* Si hay latas en _varios_ sitios adyacentes, se especifica a qué sitio va
* Si no hay latas a la vista, moverse aleatoriamente

> __¿Problemas?__

Testado en $10^4$ sesiones de limpieza: _fitness score_: 346 (lejos del óptimo).

## Algoritmo genético

Después de correr el algoritmo, seleccionamos el individuo con mayor _fitness_ de la última generación. Testado en $10^4$ sesiones de limpieza. El _fitness score_ (medio) es de 483.
* casi óptimo
* mucho mayor que la estrategia _inteligente_

### Genomas

* M: estrategia _inteligente_
* G: estrategia GA evolucionada

M: 65635365625235325265635365615135315125235325215135315165635365
62523532526563536560503530502523532520503530501513531512523532
5215135315105035305025235325205035305065635356252353252656353
656151353151252353252151353151656353656252353252656353454

G: 25435515325623525105635546115133615415103415611055015005203025
62561322523503251120523330540552312550513361541506652641502665
06012264453605631520256431054354632404350334153250253251352352
045150130156213436252353223135051260513356201524514343432

* No es muy informativo
* En algunos casos vemos que coincide y tiene sentido:
   * Posición 2: “Empty Empty Empty Empty Can”
   * Acción: 5 - recoger lata

Pero en este tipo de situaciones no siempre coinciden las estrategias:


| North | South | East | West | Current |
|-------|-------|------|------|---------|
| Empty | Can   | Empty| Can  | Can     |

* La estrategia _inteligente_ determina: acción 5 - recoger lata
* La estrategia $G$ determina: acción 3 - MoveWest
    * Robbie __no__ recoge la lata!
    * Parece una mala idea
    * Pero $G$ tiene mejor _fitness_

* No se trata del efecto de genes _aislados_, si no de la __interacción__ entre ellos (igual que en genética)
* ¿Cómo afecta al _fitness_ global la interacción?
    * Nos fijamos en el __Fenotipo__: comportamiento de Robbie en cada estrategia
    
* Nos fijamos en __dos__ cambios _visibles_ entre los comportamientos

### Movimiento con pocas latas

```{figure} ./images/ga_robbie_3.png
Comportamiento de Robbie en una situación en la que hay _pocas_ latas
```

En la gráfica anterior vemos cómo actúa Robbie cuando hay _pocas_ latas en el espacio. Cuando Robbie no detecta una lata en su espacio, ni en espacios adyacentes, las estrategias indican:
* __M__: movimiento aleatorio
* __G__: _MoveEast_ hasta encontrar una lata o una pared. Después se mueve al Norte y rodea el borde de modo antihorario hasta que encuentra una lata (ver figura).

Esta estrategia __previene__ que Robbie se _choque_ con una pared (reducción del _fitness_), que sí es posible con __M__, pero además es más eficiente a la hora de encontrar latas que simplemente moviéndose de forma aleatoria.

### Estrategia con _varias_ latas

* Situación de la figura (arriba izquierda): Robbie se encuentra en una posición en la que hay una lata y también hay latas en las posiciones vecinas (este y oeste)

* __M__: coge lata, mueve oeste, coge lata. Entonces Robbie ya no ve más latas y comienza a moverse aleatoriamente hasta que se encuentre una lata.

```{figure} ./images/ga_robbie_4.png
_Cluster_ de latas: estrategia __M__
```

* __G__: __NO__ coge la lata, si no que se mueve al oeste. Coge lata. La lata que no cogió en el paso anterior es un _marcador_ que le recuerda que hay más latas al otro lado. Después continúa cogiendo todas las latas del _cluster_.
    * Robbie coge primero la lata que se encuetra más al oeste


```{figure} ./images/ga_robbie_5.png
_Cluster_ de latas: estrategia __G__
```


* Melanie no consideró este _truco_ cuando programó su algoritmo
    * Los GA suelen llegar a estrategias que no se nos ocurren
* Estrategia __knock-out__ (genética, eliminar genes para ver su funcionalidad)
    * Melanie _eliminó_ los genes que responsables de este comportamiento en la estrategia __G__, el _fitness_ descendió de 483 a 443: este truco es, en parte, responsable del éxito de __G__

### Evolución del _fitness_

```{figure} ./images/ga_robbie_5_evolution.png
Evolución del _fitness_
```

* Las primeras estrategias son muy malas
    * Algunas consisten en Robbie estrellándose repetidamente con una pared
    * Otras Robbie se queda quieto todo el rato
    * La selección de la mejor hace que se avance rápidamente
* Hay un incremento rápido
    * Las estrategias muy malas desaparecen
    * Alrededor de la generación 200 ya se han descubierto buenos _traits_ (moverse y recoger)
    * No obstante, cuando no hay muchas latas, no sabe cómo buscarlas (como estrategia __M__)
* Y una estabilización
    * Alrededor de la generación 800 ya se ha descubierto el _truco_ de dejar latas como marcadores
    * Por la generación 900 se descubre el truco de moverse en círculos ($\sim1000$)
    
## Aplicaciones reales

* Los algoritmos genéticos evolucionan a soluciones que _funcionan_
* Es difícil ver _por qué_ funcionan
* Son soluciones diferentes de las que ideamos los humanos

```{note}
 “Evolutionary algorithms are a great tool for exploring
the dark corners of design space. You show [your designs] to people with 25
years’ experience in the industry and they say ‘Wow, does that really work?’....
We frequently see evolved designs that are completely unintelligible.”
```
Jason Lohn (NASA)

* Caso [Evolved Antenna](https://en.wikipedia.org/wiki/Evolved_antenna)

```{figure} ./images/ga_antenna.jpg
Evolución de la forma de una antena (NASA)
```